In [ ]:
! pip install ampligraph==2.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
from ampligraph.datasets import load_from_csv
# Uncomment the below block if the files 'triples_train.csv' and 'triples_test.csv' do not exist
'''
from ampligraph.evaluation import train_test_split_no_unseen
dataset = load_from_csv('/content/', 'triples.txt')
dataset_train, dataset_test = train_test_split_no_unseen(dataset, test_size = 0.3)
with open('triples_train.txt', mode = 'w') as train_file:
  for entry in dataset_train:
    train_file.write(entry[0] + "\t" + entry[1] + "\t" + entry[2] + "\n")

with open('triples_test.txt', mode = 'w') as test_file:
  for entry in dataset_test:
    test_file.write(entry[0] + "\t" + entry[1] + "\t" + entry[2] + "\n")
'''
# Comment the below two lines if the files 'triples_train.csv' and 'triples_test.csv' do not exist
dataset_train = load_from_csv('/content/', 'triples_train.txt')
dataset_test = load_from_csv('/content/', 'triples_test.txt')

In [ ]:
# # Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# # you can continue training from where you left after restoring the model
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./transe_train_logs')

# # create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=5,
                                   k=300,
                                   scoring_type='TransE')

# # you can either use optimizers/regularizers/loss/initializers with default values or you can 
# # import it and customize the hyperparameters and pass it to compile

# # Let's create an adam optimizer with customized learning rate =0.005
# # adam = tf.keras.optimizers.Adam(learning_rate=0.005)
# # Let's compile the model with self_advarsarial loss of default parameters
model.compile(optimizer='adam', loss='self_adversarial')

# # fit the model to data.
# model.fit(dataset['train'],
#              batch_size=10000,
#              epochs=10,
#              callbacks=[tensorboard_callback])

model.fit(dataset_train, batch_size = 16, epochs = 10)

# # the training can be visualised using the following command:
# # tensorboard --logdir='./transe_train_logs' --port=8891 
# # open the browser and go to the following URL: http://127.0.0.1:8891/

Epoch 1/10
79/79 [==============================] - 4s 52ms/step - loss: 259.8446
Epoch 2/10
79/79 [==============================] - 1s 6ms/step - loss: 203.3035
Epoch 3/10
79/79 [==============================] - 1s 6ms/step - loss: 163.0937
Epoch 4/10
79/79 [==============================] - 1s 7ms/step - loss: 134.1717
Epoch 5/10
79/79 [==============================] - 1s 8ms/step - loss: 113.1062
Epoch 6/10
79/79 [==============================] - 1s 7ms/step - loss: 97.4750
Epoch 7/10
79/79 [==============================] - 1s 8ms/step - loss: 85.5642
Epoch 8/10
79/79 [==============================] - 1s 7ms/step - loss: 76.2966
Epoch 9/10
79/79 [==============================] - 1s 13ms/step - loss: 68.9118
Epoch 10/10
79/79 [==============================] - 1s 13ms/step - loss: 62.9029


In [ ]:
# pred = model.predict(dataset_test, 
#                        batch_size=100)

In [ ]:
# pred

In [ ]:
# # evaluate on the test set
# ranks = model.evaluate(dataset_test,     # test set
#                        batch_size=100,      # evaluation batch size
#                        corrupt_side='s,o',   # sides to corrupt for scoring and ranking
#                        use_filter = {'train': dataset_train,
#                                       'test': dataset_test}
#                        )

# # import the evaluation metrics
# from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

# print('MR:', mr_score(ranks))
# print('MRR:', mrr_score(ranks))
# print('hits@1:', hits_at_n_score(ranks, 1))
# print('hits@10:', hits_at_n_score(ranks, 10))

In [ ]:
from ampligraph.evaluation.metrics import mr_score, mrr_score, hits_at_n_score
rank_head = model.evaluate(dataset_test, 
                           batch_size = 100,
                           corrupt_side = 's',
                           use_filter = {'train' : dataset_train,
                                         'test' :  dataset_test}
                           )
print("MR:", mr_score(rank_head))
print("MRR:", mrr_score(rank_head))
print("Hits@1:", hits_at_n_score(rank_head, 1))
print("Hits@3:", hits_at_n_score(rank_head, 3))
print("Hits@5:", hits_at_n_score(rank_head, 5))
print("Hits@10:", hits_at_n_score(rank_head, 10))
print(rank_head)

7/7 [==============================] - 9s 1s/step
MR: 108.60984848484848
MRR: 0.21344923607321992
Hits@1: 0.003787878787878788
Hits@3: 0.38446969696969696
Hits@5: 0.4772727272727273
Hits@10: 0.5454545454545454
[[307]
 [ 25]
 [  2]
 [  3]
 [  2]
 [327]
 [  4]
 [309]
 [ 56]
 [  2]
 [ 56]
 [  7]
 [146]
 [  2]
 [136]
 [132]
 [ 99]
 [114]
 [ 84]
 [325]
 [  3]
 [  2]
 [294]
 [ 69]
 [  3]
 [  2]
 [  2]
 [  2]
 [ 48]
 [  2]
 [ 16]
 [ 72]
 [179]
 [ 11]
 [  2]
 [  6]
 [  9]
 [302]
 [  5]
 [ 64]
 [  9]
 [  2]
 [353]
 [  2]
 [111]
 [162]
 [  1]
 [  5]
 [  2]
 [ 11]
 [  3]
 [  3]
 [  2]
 [608]
 [  2]
 [  5]
 [  3]
 [  2]
 [  2]
 [  4]
 [  5]
 [120]
 [  4]
 [  2]
 [338]
 [154]
 [219]
 [  2]
 [  2]
 [  2]
 [  2]
 [ 31]
 [  2]
 [  2]
 [ 25]
 [  2]
 [  2]
 [  2]
 [  4]
 [  1]
 [  2]
 [  3]
 [  2]
 [  3]
 [  3]
 [ 73]
 [  3]
 [241]
 [  3]
 [  2]
 [787]
 [  2]
 [723]
 [  6]
 [  2]
 [234]
 [181]
 [407]
 [ 42]
 [603]
 [332]
 [  3]
 [  4]
 [  2]
 [787]
 [147]
 [  6]
 [  2]
 [  5]
 [292]
 [  3]
 [313]
 [244]

In [ ]:
from ampligraph.evaluation.metrics import mr_score, mrr_score, hits_at_n_score
rank_tail = model.evaluate(dataset_test, 
                           batch_size = 100,
                           corrupt_side = 'o',
                           use_filter = {'train' : dataset_train,
                                         'test' :  dataset_test}
                           )
print("MR:", mr_score(rank_tail))
print("MRR:", mrr_score(rank_tail))
print("Hits@1:", hits_at_n_score(rank_tail, 1))
print("Hits@3:", hits_at_n_score(rank_head, 3))
print("Hits@5:", hits_at_n_score(rank_head, 5))
print("Hits@10:", hits_at_n_score(rank_tail, 10))
print(rank_tail)

7/7 [==============================] - 3s 442ms/step
MR: 104.71780303030303
MRR: 0.2202356299068447
Hits@1: 0.003787878787878788
Hits@3: 0.38446969696969696
Hits@5: 0.4772727272727273
Hits@10: 0.5852272727272727
[[ 65]
 [ 37]
 [  3]
 [  2]
 [  2]
 [  6]
 [ 10]
 [700]
 [617]
 [  2]
 [327]
 [  7]
 [452]
 [  2]
 [449]
 [169]
 [575]
 [ 27]
 [268]
 [243]
 [  2]
 [  4]
 [139]
 [114]
 [  6]
 [  2]
 [  3]
 [ 10]
 [656]
 [  2]
 [  2]
 [362]
 [169]
 [  9]
 [  2]
 [  2]
 [ 37]
 [303]
 [  3]
 [407]
 [  4]
 [  3]
 [ 51]
 [  2]
 [197]
 [457]
 [  1]
 [  2]
 [  6]
 [  7]
 [  2]
 [  4]
 [  3]
 [ 37]
 [  2]
 [  3]
 [ 13]
 [  2]
 [  2]
 [  2]
 [  2]
 [270]
 [  6]
 [  2]
 [340]
 [ 44]
 [116]
 [  3]
 [  3]
 [  2]
 [  2]
 [ 60]
 [  2]
 [  2]
 [193]
 [  6]
 [  4]
 [  2]
 [  2]
 [  1]
 [  3]
 [  8]
 [  4]
 [  6]
 [  2]
 [ 86]
 [  2]
 [206]
 [  3]
 [  3]
 [760]
 [  2]
 [ 22]
 [  3]
 [  2]
 [ 15]
 [399]
 [665]
 [  7]
 [ 43]
 [323]
 [  2]
 [  2]
 [  4]
 [763]
 [135]
 [  5]
 [  3]
 [  3]
 [151]
 [  2]
 [308]
 [62

In [ ]:
import pandas as pd
import string
sonnet_df = pd.read_csv("Sonnet_pairs.csv")
def get_head_tail(intersection_words, line_text, intersection_count):
  if intersection_count != 2:
    return None
  intersection_words_list = intersection_words.split('|')
  if (line_text.find(intersection_words_list[0]) 
        < line_text.find(intersection_words_list[1])):
    head = intersection_words_list[0]
    tail = intersection_words_list[1]
  else:
    head = intersection_words_list[1]
    tail = intersection_words_list[0]
  return (head, tail)
sonnet_df['Head_tail'] = sonnet_df.apply(lambda x : get_head_tail(x.Intersection_Words, 
                                                    x.LineText,
                                                    x.Intersection_count), axis = 1)
sonnet_df

,Unnamed: 0,LineText,Intersection_Words,Intersection_count,Head_tail
0,1,from fairest creatures we desire increase,increase|desire,2,"(desire, increase)"
1,6,feed st thy light s flame with self substantia...,feed|light,2,"(feed, light)"
2,9,thou that art now the world s fresh ornament,art|world,2,"(art, world)"
3,25,if thou couldst answer this fair child of mine,answer|child,2,"(answer, child)"
4,29,and see thy blood warm when thou feel st i...,cold|feel|blood,3,None
...,...,...,...,...,...
281,2285,which borrow d from this holy fire of love,love|fire,2,"(fire, love)"
282,2289,but at my mistress eye love s brand new fired,love|eye,2,"(eye, love)"
283,2290,the boy for trial needs would touch my breast,touch|breast|trial|boy,4,None
284,2305,which from love s fire took heat perpetual,love|fire,2,"(love, fire)"


In [ ]:
# from ampligraph.discovery import discover_facts
# from ampligraph.discovery import query_topn

# # discover_facts(dataset, 
# #                model, 
# #                top_n=50, 
# #                strategy='entity_frequency', 
# #                max_candidates=100,
# #                target_rel= ['clear', 'brilliant'], 
# #                seed=0)
# query_topn(model, top_n=10,
#            head='time', relation=None, tail='money',
#            ents_to_consider=None, rels_to_consider=None)

from ampligraph.discovery import query_topn
# for row in sonnet_df.itertuples(index = False):
#   if row.Head_tail:
#     print(query_topn(model, top_n = 10, head = row.Head_tail[0], 
#                      relation = None,
#                      tail = row.Head_tail[1]))
    
def get_topn_relations(model, n, head_tail):
  if not head_tail:
    return None
  relations = []
  arr = query_topn(model, top_n = n, head = head_tail[0], 
             relation = None,
             tail = head_tail[1])
  for triple in arr[0]:
    relations.append(triple[1])
  return relations

sonnet_df['Relations'] = sonnet_df.apply(lambda x : get_topn_relations(model, 10, 
                                                                  x.Head_tail),
                                         axis = 1)
# arr = query_topn(model, 10, head = "time", relation = None, tail = "money")
# relations = []
# for triple in arr[0]:
#   relations.append(triple[1])
# print(relations)



In [ ]:
sonnet_df.to_csv("Sonnet_relations.csv")

In [ ]:
len(dataset_test)

528

In [ ]:
# ranks